In [ ]:
from library.utils.jordan_wigner import (
    FH_zero_particle_state,
    c,
    c_dag,
    n_op,
    full_n_op,
    density_matrix_size_bytes,
    embed,
    emulated_FH_Hamiltonian,
    FH_energy_shift
)
from feedback_grape.utils.operators import (
    sigmax,
    sigmay,
    sigmaz,
    sigmap,
    sigmam,
    identity,
)
from feedback_grape.utils.states import basis
from feedback_grape.utils.tensor import tensor
from feedback_grape.grape import optimize_pulse
from jax import numpy as jnp

"""
    Test for emulating Fermi Hubbard model with Jordan-Wigner transformation and
    for preparing zero fermionic particle state from thermal state using grape in system without dissipation.
"""

# Initialize spin J = 0, 0.5 Fermi Hubbard model on n sites
n  = 2   # e^- sites
J  = 0.5 # Spin
mu = 1.0 # Chemical potential
U  = 2.0 # On-site interaction
t  = 3.0 # Transfer parameter

# Evolution parameters
n_ts     = 10 # Number of time slots
evo_time = 2  # Total evolution time

# Initialize thermal state at inverse temperature gamma
gamma = 1.0

if J == 0.0:   spins = [0.0]
elif J == 0.5: spins = [-0.5, 0.5]
else: raise ValueError("J must be either 0 or 1/2.")

phi  = basis(2, 1) + jnp.exp(-gamma)*basis(2, 0)
phi /= jnp.linalg.norm(phi)
N_qubits = n * int(2*J + 1)

psi_0 = tensor(*([phi]*N_qubits)) # Initial state is thermal state on all qubits
rho_0 = psi_0 @ psi_0.conj().T # Density matrix form
psi_target = FH_zero_particle_state(n, J) # Target state is zero particle state
rho_target = psi_target @ psi_target.conj().T # Density matrix form

# Drift Hamiltonian
H_d = identity(2**N_qubits)

# Controls
Sx = sum(embed(sigmax(), j, s, n, J) for j in range(n) for s in spins)
Sy = sum(embed(sigmay(), j, s, n, J) for j in range(n) for s in spins)
# Sz = sum(embed(sigmaz(), j, s, n, J) for j in range(n) for s in spins)
H_c = [Sx, Sy]

# Optimize pulse
result = optimize_pulse(
    H_drift=H_d,
    H_control=H_c,
    U_0=rho_0,
    C_target=rho_target,
    num_t_slots=n_ts,
    total_evo_time=evo_time,
    evo_type="density",
    optimizer="adam",
    convergence_threshold=1e-16,
    max_iter=1000,
    learning_rate=0.01,
)

print("Final fidelity: ", result.final_fidelity, "iterations: ", result.iterations)
print("Number of fermions in output state: ", jnp.trace(full_n_op(n, J) @ result.final_operator))

Final fidelity:  0.9999999999999987 iterations:  302
Number of fermions in output state:  (1.3185088786615689e-15+3.7560670468002175e-17j)


In [13]:
# Now the same with dissipation such that we got energy relaxation to the thermal state
Sm = sum(embed(sigmam(), j, s, n, J) for j in range(n) for s in spins)
Sp = sum(embed(sigmap(), j, s, n, J) for j in range(n) for s in spins)

damping = 0.01
decay_op = (Sm + jnp.exp(-gamma)*Sp) * jnp.sqrt(damping) # Operator which eventually leads to thermal state
l_ops = [decay_op]

# Optimize pulse
result = optimize_pulse(
    H_drift=H_d,
    H_control=H_c,
    U_0=rho_0,
    C_target=rho_target,
    c_ops=l_ops,
    num_t_slots=n_ts,
    total_evo_time=evo_time,
    evo_type="density",
    optimizer="adam",
    convergence_threshold=1e-16,
    max_iter=1000,
    learning_rate=0.01,
)

print("Final fidelity: ", result.final_fidelity, "iterations: ", result.iterations)
print("Number of fermions in output state: ", jnp.trace(full_n_op(n, J) @ result.final_operator))

Final fidelity:  0.9924546746823857 iterations:  337
Number of fermions in output state:  (0.007790402645267169+0j)
